**installing and importing useful libraries**

In [1]:
# Standard libraries
import os
import sys
import math
import random
import datetime

# Numerical computing
import numpy as np

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
sns.set_theme()

#importing optimization techniques
from implementations import *
from cross_validation import *
from helpers import *

**importing data**

In [13]:
#IF WE USE OUTLIERS DETECTION WE SHOULD ACHANGE Y TRAIN


# Paths to the CSV files ---- > change with yours
datapath_1 = r"C:\Users\sanni\OneDrive\Desktop\POLIMI\EPFL\ML\Project_1\project-1-girl_power\data\pca_train_projection.csv"
datapath_2 = r"C:\Users\sanni\OneDrive\Desktop\POLIMI\EPFL\ML\Project_1\project-1-girl_power\data\dataset\dataset\y_train.csv"

# Load the data
x_train = np.loadtxt(datapath_1, delimiter=',', dtype=float, skiprows=1)
y_train = np.loadtxt(datapath_2, delimiter=',' , dtype=float, skiprows=1, usecols=1)

# Convert y_train from {-1, 1} to {0, 1}
y_train = np.where(y_train == -1, 0, y_train)


print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)




x_train shape: (328135, 60)
y_train shape: (328135,)


### Splitting the data between 20 % validation set and 80 % training set

In [14]:
def train_val_split(X, y, val_ratio=0.20, seed=42):
    X = np.asarray(X); y = np.asarray(y)
    n = X.shape[0]
    rng = np.random.default_rng(seed)
    idx = np.arange(n)
    rng.shuffle(idx)

    n_val = int(np.round(n * val_ratio))
    val_idx = idx[:n_val]
    train_idx = idx[n_val:]

    return X[train_idx], X[val_idx], y[train_idx], y[val_idx], train_idx, val_idx



x_train_split, x_val_split, y_train_split, y_val_split, tr_idx, va_idx = train_val_split(x_train, y_train, val_ratio=0.20, seed=42)
print(x_train_split.shape)
print(x_val_split.shape)


(262508, 60)
(65627, 60)


In [15]:
# OVERSAMPLING AND BALANCING DATA
def make_balanced_subset(x_train_filtered, y_train, majority_class=-1, minority_class=1,
                         seed_major=0, seed_minor=42, seed_shuffle=7):
    # Boolean masks
    maj_mask = (y_train == majority_class)
    min_mask = (y_train == minority_class)

    # Indices per class
    maj_idx = np.nonzero(maj_mask)[0]
    min_idx = np.nonzero(min_mask)[0]

    # Target size = size of minority (undersample majority)
    n = len(min_idx)

    # Sample without replacement
    rs_maj = np.random.RandomState(seed_major)
    rs_min = np.random.RandomState(seed_minor)
    sampled_maj = rs_maj.choice(maj_idx, size=n, replace=False)
    sampled_min = rs_min.choice(min_idx, size=n, replace=False)

    # Combine and shuffle
    balanced_idx = np.concatenate([sampled_maj, sampled_min])
    rs_shuf = np.random.RandomState(seed_shuffle)
    rs_shuf.shuffle(balanced_idx)

    # Slice arrays
    x_bal = x_train_filtered[balanced_idx]
    y_bal = y_train[balanced_idx]
    return x_bal, y_bal, balanced_idx


In [ ]:
# NORMALIZE DATA --- > not needed with pca

### Hyperparameters definition and metrics

In [ ]:
#lambdas = [ 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1]  # regularization parameters list
#gammas = [1e-4, 1e-3, 1e-2, 1e-1, 1] # step-size parameters list
#max_iters = [100, 1000, 10000] # max iters list

lambdas = [ 1e-4, 1e-3, 1e-2, 1e-1]  # regularization parameters list
gammas = [1e-4, 1e-3] # step-size parameters list
max_iters = [100] # max iters list


def compute_auc(y_true, y_scores):
    """
    AUC calculation using Mann-Whitney statistics
    Inputs : 
            - y_true : numpy array containing the real {0, 1} values of the dataset
            - y_scores : numpy array containing our predictions
    Output : 
            AUC Area under the ROC curve 
    """
    order = np.argsort(y_scores)
    y_true_sorted = y_true[order]

    n_pos = np.sum(y_true)
    n_neg = len(y_true) - n_pos

    # rank positions 
    rank_positions = np.arange(1, len(y_true_sorted) + 1)
    rank_sum = np.sum(rank_positions[y_true_sorted == 1])

    # AUC using Mann–Whitney
    auc = (rank_sum - n_pos*(n_pos+1)/2) / (n_pos * n_neg)
    return auc

def compute_accuracy(y_true, y_scores) : 
    """
    Accuracy computation
    Inputs : 
            - y_true : numpy array containing the real {0, 1} values of the dataset
            - y_scores : numpy array containing our predictions
    Output : 
            Accuracy = correct predictions / total predictions %

    """
    correct_pred = (y_true == y_scores)
    accuracy = np.mean(correct_pred) * 100
    return accuracy


### K-fold cross validation functions

In [28]:

def build_k_indices(N, k_fold, seed=21): 
    """build k indices for k-fold.

    Args:
        N:      num of samples
        k_fold: K in K-fold, i.e. the fold num
        seed:   the random seed

    Returns:
        A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold

    """
    num_row = N
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval : (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)
     

def k_fold_cross_validation(y_train, x_train, lambdas, gammas, max_iters, k_fold, methods, seed, oversampling = True) :
    # dictionary to contain the best method with the best parameters and its metrics
    best_overall = {"method": "", "lambda_": 0, "gamma": 0, "max_iter": 0, "train loss": 0, "test loss": 0, "AUC": 0, "accuracy": 0, "y_pred" : None, "w_opt": None}
    results =[] # to keep the best results per method
    #creating k folders on the train set 
    k_indices = build_k_indices(len(y_train), k_fold, seed)

    for method in methods : #scrolling methods
        best_per_method =  {"method": "", "lambda_": 0, "gamma": 0, "max_iter": 0, "train loss": 0, "test loss": 0, "AUC": 0, "accuracy": 0, "y_pred" : None, "w_opt": None} 
        for lam in lambdas : # scrolling lambdas
            for gam in gammas : #scrolling gammas
                for max_it in max_iters : #scrolling iters ----> model defined at this point
                    logistic_loss_tr = []
                    logistic_loss_te =[]
                    AUC=  []
                    accuracies = []
                    
                    for k in range(k_fold) : 
                        #k-th subgroup in test, others in train
                        test_mask = np.isin(np.arange(len(y_train)), k_indices[k, :])
                        y_test_k = y_train[test_mask]
                        x_test_k = x_train[test_mask]
    
                        y_train_k=y_train[~test_mask]
                        x_train_k=x_train[~test_mask]
                        
                        if oversampling :
                            #oversampling dataset
                            x_train_k , y_train_k, _ = make_balanced_subset(x_train_k, y_train_k, 0, 1, seed_major=0, seed_minor=42, seed_shuffle=7)




                        #train the model
                        if method == "reg_logistic_regression" :
                            w_opt, loss = reg_logistic_regression(y_train_k, x_train_k,lam, np.zeros(x_train_k.shape[1]), max_it, gam)
                        elif method == "least_squares" :
                            w_opt, loss = least_squares(y_train_k, x_train_k)
                        elif method == "adam_reg_logistic_regression":
                            w_opt, loss = reg_logistic_regression_adam(y_train_k, x_train_k, lam, np.zeros(x_train_k.shape[1]), max_it, 0.9, 0.999, gam, 700 )
                        elif method == "ridge_regression" :
                            w_opt, loss = ridge_regression(y_train_k, x_train_k, lam)

                        #computing metrics 
                        logistic_loss_tr.append(compute_logistic_loss(y_train_k, x_train_k, w_opt)) #without penalizing term
                        logistic_loss_te.append(compute_logistic_loss(y_test_k, x_test_k, w_opt)) #without penalizing term CAPIRE CHE SENSO HA COMPARARE STE LOSS

                        if method in ["adam_reg_logistic_regression", "reg_logistic_regression"]:
                            pred = sigmoid(x_test_k @ w_opt) 
                        else:
                            pred = x_test_k @ w_opt

                        AUC.append(compute_auc(y_test_k, pred))
                        accuracies.append(compute_accuracy(y_test_k, (pred >= 0.5).astype(int)))
                    # updating    
                    if np.mean(AUC) > best_per_method["AUC"]:
                        best_per_method.update({"method": method, "gamma": gam, "lambda_": lam, "max_iter": max_it, "train loss": np.mean(logistic_loss_tr), "test loss": np.mean(logistic_loss_te), "AUC": np.mean(AUC), "accuracy": np.mean(accuracies), "y_pred" : (pred >= 0.5).astype(int) , "w_opt": w_opt})

                    if np.mean(AUC) > best_overall["AUC"]:
                        best_overall.update({"method": method, "gamma": gam, "lambda_": lam, "max_iter": max_it, "train loss": np.mean(logistic_loss_tr), "test loss": np.mean(logistic_loss_te), "AUC": np.mean(AUC), "accuracy": np.mean(accuracies), "y_pred" : (pred >= 0.5).astype(int) , "w_opt": w_opt})

        results.append(best_per_method)
        print(f"For method {method}, best λ={best_per_method['lambda_']}, γ={best_per_method['gamma']}, max_iter={best_per_method['max_iter']}")


    
    print(f"The best method is {best_overall['method']}, with best λ={best_overall['lambda_']}, γ={best_overall['gamma']}, max_iter={best_overall['max_iter']}")



    return best_overall, results 

        
methods=["reg_logistic_regression", "least_squares", "adam_reg_logistic_regression", "ridge_regression"]
best_method, results = k_fold_cross_validation(y_train_split, x_train_split, lambdas, gammas, max_iters, 5, methods, seed = 21, oversampling=False)






For method reg_logistic_regression, best λ=0.0001, γ=0.001, max_iter=100
For method least_squares, best λ=0.0001, γ=0.0001, max_iter=100
For method adam_reg_logistic_regression, best λ=0.01, γ=0.001, max_iter=100
For method ridge_regression, best λ=0.01, γ=0.0001, max_iter=100
The best method is ridge_regression, with best λ=0.01, γ=0.0001, max_iter=100


In [29]:
print(best_method) 
print(results)

{'method': 'ridge_regression', 'lambda_': 0.01, 'gamma': 0.0001, 'max_iter': 100, 'train loss': 0.6817317752542131, 'test loss': 0.6817415970302714, 'AUC': 0.8489835054683796, 'accuracy': 91.2089293537266, 'y_pred': array([0, 0, 0, ..., 0, 0, 0]), 'w_opt': array([ 3.50058611e-02, -8.87536146e-03,  1.65235944e-03, -5.91319982e-04,
        4.64029179e-03,  1.18368460e-02, -1.03447114e-02, -1.21375923e-02,
       -2.11234177e-02, -3.95062146e-04,  1.01048533e-02, -6.49793695e-03,
        2.72864303e-03,  2.01904766e-02, -5.96512952e-03, -1.69934947e-02,
       -6.66632778e-03,  7.57231662e-03,  7.50320969e-03,  7.34997888e-03,
        1.57390387e-02, -9.80302288e-04, -5.27560938e-04, -5.84386092e-03,
        4.65541021e-03, -9.00916660e-03,  6.43199674e-03, -1.06177109e-02,
       -6.08629998e-03,  4.28472084e-03, -2.26993426e-03, -8.33775882e-04,
        6.86705614e-03,  2.30045709e-03, -1.38317554e-03, -5.93250408e-03,
        5.17540744e-03,  1.39433428e-03, -1.26820932e-03,  1.0824158

### Validation on our validation sample

In [30]:
def validation(w_opt, x_val, y_val): 
    w_opt = best_method["w_opt"]
    if best_method["method"] in ["reg_logistic_regression", "adam_reg_logistic_regression"] :
        predictions = sigmoid(x_val @ w_opt) 
    else:
        predictions = x_val @ w_opt
    AUC = compute_auc(y_val, predictions)
    accuracy = compute_accuracy(y_val, (predictions>=0.5).astype(int))

    return AUC, accuracy


AUC, accuracy = validation(best_method["w_opt"], x_val_split, y_val_split) 

print(f"the best method {best_method['method']} has an accuracy = {accuracy} and an AUC = {AUC} on our validation set")

the best method ridge_regression has an accuracy = 91.3069315982751 and an AUC = 0.8464946861089571 on our validation set


In [ ]:


cross_validation_visualization(param_grid, logistic_loss_tr, logistic_loss_te)

num_par = len(param_grid)
w = 0.3 # bar width
pos = np.arange(num_par)
plt.bar(pos - w, AUC, width = w, label='AUC' )
plt.bar(pos, accuracies, width=w, label= 'Accuracy')
plt.bar(pos + w, logistic_loss_tr, width = w, label = 'train logistic loss' )
plt.bar(pos + w, logistic_loss_te, width=w, label= "test logistic loss" )

plt.xticks(pos, param_grid)
plt.xlabel('Different regularization hyperparameter values')
plt.title('Finding the best regularization hyperparamter - ADAM case')
plt.legend()

plt.show()

AUC calcolata manualmente: 0.64
AUC con scikit-learn: 0.64
Differenza: 0.0


Now that we have found the best regularization hyperparameter for Adam reg log and GD reg log, let's test which model is the best to make predictions using a k-fold cross validation. 

In [ ]:
##### CONFRONTING ALL THE MODELS WITH K-FOLD CROSS VALIDATION 




N = len(y_train)
d = x_train.shape[1]
k = 10  # folds
initial_w = np.zeros(d, )
max_iters = 10000
gamma = 0.01
beta_1 = 0.9 #using Adam paper as benchmark
beta_2 = 0.999 #using Adam paper as benchmark
mini_batch_size = 700 #using Adam paper as benchmark 

models = [
    ("MSE GD", lambda: mean_squared_error_gd(initial_w=initial_w, max_iters=max_iters, gamma=gamma)),
    ("MSE SGD", lambda: mean_squared_error_sgd(initial_w=initial_w, max_iters=max_iters, gamma=gamma, mini_batch_size=mini_batch_size)),
    ("Least Squares", lambda: least_squares()),
    ("Ridge Regression", lambda: ridge_regression()), #understand which lambda here 
    ("Logistic Regression GD", lambda: logistic_regression(initial_w=initial_w, max_iters=max_iters, gamma=gamma)),
    ("Reg Logistic ADAM", lambda: reg_logistic_regression_adam(lambda_adam, initial_w=initial_w, max_iters=max_iters,  beta_1=beta_1, beta_2=beta_2,gamma=gamma, mini_batch_size=mini_batch_size)),
    ("Reg Logistic GD", lambda: reg_logistic_regression(lambda_gd, initial_w=initial_w, max_iters=max_iters, gamma=gamma))
]  ### does it make sense to include all the models ????


#### validation metrics
logistic_loss_tr = []
logistic_loss_te =[]
AUC=  []
accuracies = []

####  k-fold cross validation
k_indices =build_k_indices(N, k_fold, seed)
for model in models:
    fold_loss_tr = 0
    fold_loss_te = 0
    fold_AUC = 0
    fold_accuracy = 0

    for k in range(k_fold):
        #k-th subgroup in test, others in train
        test_mask = np.isin(np.arange(len(y)), k_indices[k, :])
        y_test_k = y_train[test_mask]
        x_test_k = x_train[test_mask]
    
        y_train_k=y_train[~test_mask]
        x_train_k=x_train[~test_mask]
    
        # train the model
        name, model_fn = model
        w_opt, loss_tr = model_fn(y_train_k, x_train_k)
    
        # calculate the loss for test data
        loss_te = compute_logistic_loss(y_test_k, x_test_k, w_opt)

        # compute AUC 
        predictions = sigmoid (x_test_k @ w_opt)
        AUC = compute_auc(y_test_k, predictions)

        # compute accuracy
        y_pred = (predictions >= 0.5).astype(int)
        accuracy = np.mean(y_pred == y_test)
       

        # storing validation results
        fold_loss_tr += loss_tr
        fold_loss_te+=loss_te
        fold_AUC += AUC
        fold_accuracy += accuracy
    logistic_loss_tr.append(fold_loss_tr/k_fold)
    logistic_loss_te.append(fold_loss_te/k_fold)
    AUC.append(fold_AUC/k_fold)
    accuracies.append(fold_accuracy/k_fold)





#### Plotting results per model   

num_models = len(models)
w = 0.3
pos = np.arange(num_models)
labels = [name for name, _ in models]

plt.bar(pos - w, AUC, width=w, label='AUC')
plt.bar(pos, accuracies, width=w, label='Accuracy')
plt.bar(pos + w, logistic_loss_te, width=w, label='test logistic loss')
plt.bar(pos + 2*w, logistic_loss_tr, width=w, label='train logistic loss')

plt.xticks(pos, labels, rotation=45)
plt.xlabel('Different models')
plt.title('Finding the best model')
plt.legend()
plt.show()

     




